In [ ]:
%%capture
!pip install bnlp-toolkit
!pip install transformers
!pip install git+https://github.com/csebuetnlp/normalizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

import torch
from torch.utils.data import DataLoader, TensorDataset
from torchtext.vocab import Vocab
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn as nn

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, AutoTokenizer, AutoModelForSequenceClassification, PreTrainedTokenizerFast
from normalizer import normalize


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

from bnlp import NLTKTokenizer
import math
import random

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
rand_seed = 1601
np.random.seed(rand_seed)
random.seed(rand_seed)
torch.manual_seed(rand_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(rand_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
df_train = pd.read_excel('/content/drive/Shareddrives/TFQ/SentNoB_After_Def/noise_train.xlsx')
df_test = pd.read_excel('/content/drive/Shareddrives/TFQ/SentNoB_After_Def/noise_test.xlsx')
df_val = pd.read_excel('/content/drive/Shareddrives/TFQ/SentNoB_After_Def/noise_val.xlsx')

In [ ]:
col_name = ['Slang', 'Local', 'Word Misuse', 'Context/Word Missing', 'Wrong Serial',
            'Mixed Language', 'Punctuation', 'Space Error', 'Spelling', 'Internet Garbage']

for col in col_name:
    df_train[col] = pd.to_numeric(df_train[col], errors='coerce')
    df_test[col] = pd.to_numeric(df_test[col], errors='coerce')
    df_val[col] = pd.to_numeric(df_val[col], errors='coerce')

df_train = df_train.fillna(0)
df_test = df_test.fillna(0)
df_val = df_val.fillna(0)

In [ ]:
df_train.head(5)

,0,Data,Slang,Local,Word Misuse,Context/Word Missing,Wrong Serial,Mixed Language,Punctuation,Space Error,Spelling,Internet Garbage
0,0,মুগ্ধ হয়ে গেলাম মামু. আর তোমায় কি কমু. বলো তোম...,0.0,1,0,0,0,0,1.0,0,0.0,0
1,1,এই কুত্তার বাচ্চাদের জন্য দেশটা আজ এমন অবস্তায়...,1.0,0,0,0,0,0,1.0,0,1.0,0
2,2,ভাই আপনার কথাই যাদু রয়েছে,0.0,0,1,0,0,0,0.0,0,0.0,0
3,3,উওরটা আমার অনেক ভাল লেগেছে,0.0,0,1,0,0,0,0.0,0,1.0,0
4,4,আমার নিজের গাড়ী নিয়ে কি সাজেক যেতে পারবো না ?...,0.0,0,0,0,0,1,0.0,0,0.0,0


In [ ]:
X_train = df_train['Data'].values
y_train = df_train[col_name].values

X_test = df_test['Data'].values
y_test = df_test[col_name].values

X_val = df_val['Data'].values
y_val = df_val[col_name].values

In [ ]:
model_path = 'sagorsarker/bangla-bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=10)

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/660M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt',
            return_token_type_ids=False,
            return_attention_mask=True,
            return_overflowing_tokens=False,
            return_special_tokens_mask=False,
            truncation=True,
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': label
        }

In [ ]:
batch_size = 128
max_length = 110
train_dataset = CustomDataset(X_train, y_train, tokenizer, max_length=max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = CustomDataset(X_val, y_val, tokenizer, max_length=max_length)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

test_dataset = CustomDataset(X_test, y_test, tokenizer, max_length=max_length)
test_dataloader = DataLoader(test_dataset, batch_size=1)

In [ ]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00001)

In [ ]:
device = 'cuda:0'
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(102025, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
def train_model():
  threshold = 0.5

  total_loss = 0
  total_samples = 0
  total_correct = 0
  model.train()

  for idx, batch in enumerate(train_dataloader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = (batch['label'] > 0).to(device, dtype=torch.float)
    outputs = model(input_ids, attention_mask=attention_mask)
    outputs = torch.sigmoid(outputs.logits)
    # print(outputs)
    # print(labels)
    loss = criterion(outputs, labels)
    # print(loss)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    total_loss += loss.item()

    predictions = outputs >= threshold
    correct_predictions = (predictions == labels).all(dim=1)
    total_correct += correct_predictions.sum().item()
    total_samples += labels.size(0)

  avg_loss = total_loss / len(train_dataloader)
  accuracy = total_correct / total_samples

  print(f'Train Loss: {avg_loss}, Train Accuracy: {accuracy}')

  total_loss = 0
  total_samples = 0
  total_correct = 0
  model.eval()

  for idx, batch in enumerate(val_dataloader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = (batch['label'] > 0).to(device, dtype=torch.float)

    outputs = model(input_ids, attention_mask=attention_mask)
    outputs = torch.sigmoid(outputs.logits)
    loss = criterion(outputs, labels)
    total_loss += loss.item()

    predictions = outputs >= threshold
    correct_predictions = (predictions == labels).all(dim=1)
    total_correct += correct_predictions.sum().item()
    total_samples += labels.size(0)

  avg_loss = total_loss / len(val_dataloader)
  accuracy = total_correct / total_samples

  print(f'Valid Loss: {avg_loss}, Valid Accuracy: {accuracy}')

In [ ]:
# OUTPUT OF THIS CELL IS STORED MANUALLY IN THE NEXT CELL FOR FUTURE USE
# for i in range(0, 25):
#   print(i + 1)
#   train_model()
#   torch.save(model.state_dict(), f'/content/drive/Shareddrives/TFQ/SentNoB_After_Def/BERT_CheckPoints/BERT_{i+1}')

1<br>
Train Loss: 0.36378707539849947, Train Accuracy: 0.14167545211256183<br>
Valid Loss: 0.3587777341405551, Valid Accuracy: 0.09330628803245436<br>
2<br>
Train Loss: 0.30209597111365954, Train Accuracy: 0.22536696131700593<br>
Valid Loss: 0.3323027913769086, Valid Accuracy: 0.13116970926301555<br>
3<br>
Train Loss: 0.27542460126889184, Train Accuracy: 0.2771875760278972<br>
Valid Loss: 0.32612991022566956, Valid Accuracy: 0.14739688979039892<br>
4<br>
Train Loss: 0.2543615512742897, Train Accuracy: 0.3204930662557781<br>
Valid Loss: 0.31523770466446877, Valid Accuracy: 0.17308992562542258<br>
5<br>
Train Loss: 0.23574038189618698, Train Accuracy: 0.3641229421782499<br>
Valid Loss: 0.30427372455596924, Valid Accuracy: 0.19810682893847195<br>
6<br>
Train Loss: 0.21805866062641144, Train Accuracy: 0.4027248398345633<br>
Valid Loss: 0.3206155262887478, Valid Accuracy: 0.19743069641649763<br>
7<br>
Train Loss: 0.19904563711097203, Train Accuracy: 0.4492741870083529<br>
Valid Loss: 0.3048423652847608, Valid Accuracy: 0.23799864773495605<br>
8<br>
Train Loss: 0.1802117508773359, Train Accuracy: 0.4964723055713243<br>
Valid Loss: 0.3303120005875826, Valid Accuracy: 0.22041920216362407<br>
9<br>
Train Loss: 0.16087630136334216, Train Accuracy: 0.553320898548374<br>
Valid Loss: 0.3410836861779292, Valid Accuracy: 0.22920892494929007<br>
10<br>
Train Loss: 0.14484004672481607, Train Accuracy: 0.5959776173870732<br>
Valid Loss: 0.34704508384068805, Valid Accuracy: 0.2434077079107505<br>
11<br>
Train Loss: 0.12991042632049848, Train Accuracy: 0.6353904792798638<br>
Valid Loss: 0.35285786104698974, Valid Accuracy: 0.25287356321839083<br>
12<br>
Train Loss: 0.11628167508797324, Train Accuracy: 0.6789392587786879<br>
Valid Loss: 0.371913630515337, Valid Accuracy: 0.2434077079107505<br>
13<br>
Train Loss: 0.10703817511778421, Train Accuracy: 0.6985645933014354<br>
Valid Loss: 0.3764635603874922, Valid Accuracy: 0.24611223799864773<br>
14<br>
Train Loss: 0.0952275443455407, Train Accuracy: 0.7368421052631579<br>
Valid Loss: 0.3916153845687707, Valid Accuracy: 0.2630155510480054<br>
15<br>
Train Loss: 0.0886762140058174, Train Accuracy: 0.7576838861406212<br>
Valid Loss: 0.39439228797952336, Valid Accuracy: 0.2657200811359026<br>
16<br>
Train Loss: 0.08231052516964434, Train Accuracy: 0.7735787851755738<br>
Valid Loss: 0.40844352046648663, Valid Accuracy: 0.25625422582826235<br>
17<br>
Train Loss: 0.07759806394113777, Train Accuracy: 0.7863109236882654<br>
Valid Loss: 0.41853562618295354, Valid Accuracy: 0.2589587559161596<br>
18<br>
Train Loss: 0.07157390049316105, Train Accuracy: 0.8039899440434677<br>
Valid Loss: 0.4191291642685731, Valid Accuracy: 0.2691007437457742<br>
19<br>
Train Loss: 0.06756734965764796, Train Accuracy: 0.8161544075906253<br>
Valid Loss: 0.4379264290134112, Valid Accuracy: 0.2711291413116971<br>
20<br>
Train Loss: 0.06322792535309965, Train Accuracy: 0.8249128213445787<br>
Valid Loss: 0.45910438522696495, Valid Accuracy: 0.25760649087221094<br>
21<br>
Train Loss: 0.06014722289392059, Train Accuracy: 0.8373205741626795<br>
Valid Loss: 0.441945963849624, Valid Accuracy: 0.2657200811359026<br>
22<br>
Train Loss: 0.057565186307837926, Train Accuracy: 0.8404022382612927<br>
Valid Loss: 0.46982312326629955, Valid Accuracy: 0.25693035835023664<br>
23<br>
Train Loss: 0.05423976226174152, Train Accuracy: 0.8517557375719731<br>
Valid Loss: 0.47728507220745087, Valid Accuracy: 0.24475997295469912<br>
24<br>
Train Loss: 0.052984909728709895, Train Accuracy: 0.8571081015327224<br>
Valid Loss: 0.47935011982917786, Valid Accuracy: 0.25693035835023664<br>
25<br>
Train Loss: 0.05063420280312318, Train Accuracy: 0.8600275727840402<br>
Valid Loss: 0.4851290745039781, Valid Accuracy: 0.2501690331304936<br>

In [ ]:
def test_model():
  model.eval()

  all_predictions = []
  all_labels = []

  total_correct = 0
  total_samples = 0

  for idx, batch in enumerate(test_dataloader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = (batch['label'] > 0).to(device, dtype=torch.float)

    outputs = model(input_ids, attention_mask=attention_mask)
    outputs = torch.sigmoid(outputs.logits)

    predictions = outputs >= 0.5

    correct_predictions = (predictions == labels).all(dim=1)
    total_correct += correct_predictions.sum().item()
    total_samples += labels.size(0)

    all_predictions.append(predictions.cpu().numpy())
    all_labels.append(labels.cpu().numpy())

  all_predictions_np = np.concatenate([p for p in all_predictions])
  all_labels_np = np.concatenate([l for l in all_labels])

  accuracy = total_correct / total_samples
  precision = precision_score(all_labels_np, all_predictions_np, average='micro')
  recall = recall_score(all_labels_np, all_predictions_np, average='micro')
  f1 = f1_score(all_labels_np, all_predictions_np, average='micro')

  print(f'Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}')
  return all_predictions_np, all_labels_np

In [ ]:
model.load_state_dict(torch.load(f'/content/drive/Shareddrives/TFQ/SentNoB_After_Def/BERT_CheckPoints/BERT_19'))
y_pred, y_true = test_model()

Accuracy: 0.2570659488559892, Precision: 0.7290708371665133, Recall: 0.5429941760877013, F1-Score: 0.6224229334380522


In [ ]:
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.76      0.76      0.76       127
           1       0.46      0.49      0.47       152
           2       0.65      0.16      0.25        82
           3       0.33      0.06      0.10        50
           4       0.00      0.00      0.00         4
           5       0.75      0.85      0.80       538
           6       0.83      0.54      0.65       881
           7       0.86      0.21      0.33       429
           8       0.64      0.55      0.59       609
           9       0.82      0.89      0.86        47

   micro avg       0.73      0.54      0.62      2919
   macro avg       0.61      0.45      0.48      2919
weighted avg       0.74      0.54      0.60      2919
 samples avg       0.64      0.52      0.55      2919



In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

In [ ]:
multilabel_confusion_matrix(y_true, y_pred)

array([[[1328,   31],
        [  31,   96]],

       [[1248,   86],
        [  78,   74]],

       [[1397,    7],
        [  69,   13]],

       [[1430,    6],
        [  47,    3]],

       [[1482,    0],
        [   4,    0]],

       [[ 797,  151],
        [  81,  457]],

       [[ 505,  100],
        [ 405,  476]],

       [[1043,   14],
        [ 340,   89]],

       [[ 692,  185],
        [ 274,  335]],

       [[1430,    9],
        [   5,   42]]])